In [20]:
from typing import Callable
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier


def _prior(df, household_idx=1):
    if household_idx in [1, 3]:
        df["preds_time"] = df.apply(lambda row: 0 if (row.Time >= 18) & (row.Time <= 40)  else 1)
    elif household_idx in [2]:
        df["preds_time"] = df.apply(lambda row: 0 if (row.Time >= 24) & (row.Time <= 28) else 1)

def multiview_iteration(df_t:pd.DataFrame, prior: Callable, max_iter:int=100):
    # line1
    df_t = _prior(df_t)
    df_t["preds_t"] = df_t["preds_time"]
    
    stop_condition = False
    t = 1
    eta_1 = 0.5
    
    while (not stop_condition)&(t < max_iter):
        df_new = pd.DataFrame()
        
        # line2-5
        model_energy = DecisionTreeClassifier()
        model_energy.fit(df_t.Energy, df_t.preds)
        df_t["preds_energy"] = model_energy(df_t.Energy)
        
        # line6
        df_t["preds_majority_vote"] = df_t.preds_energy & df_t.preds_time
        
        # line 7
        df_new["preds_new"] = df_t[df_t.preds_t != df_t.preds_majority_vote]
        
        # line 8
        df_tplus1 = df_t[df_t.preds_t == df_new.preds_new]
        synmetric_diff = _synmetric_diff_sample(df_t, df_new)
        # TODO: Implement
        df_tplus1 = pd.concat([df_tplus1, synmetric_diff], axis=0)
        # TODO: Implement
        
def _synmetric_diff_sample():
    pass
# TODO: Implement